# Imports

In [1]:
!pip install python-dotenv

In [2]:
import requests
from kafka import KafkaProducer
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

In [3]:
# Charger les variables d'environnement depuis le fichier .env copié dans le conteneur
load_dotenv('/home/jovyan/.env')

# Configuration de l'API OpenSky
OPENSKY_ALL_FLIGHTS_URL = "https://opensky-network.org/api/flights/all?begin={}&end={}"
USERNAME = os.environ.get('OPENSKY_USERNAME')
PASSWORD = os.environ.get('OPENSKY_PASSWORD')

# Fonction pour envoyer les données OpenSky à Kafka

In [4]:

def send_opensky_to_kafka(topic, url, username, password):
    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from OpenSky
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        data = response.json()
        if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
            flights = [item for item in data if 'icao24' in item]
        else:
            flights = []


        # Publish each state to Kafka
        for flight in flights:
            producer.send(topic, value=flight)
            #print(f"Sent: {flight}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(flights)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [5]:
flights = {
    "icao24": "icao24",
    "firstSeen": "firstSeen",
    "estDepartureAirport": "estDepartureAirport",
    "lastSeen": "lastSeen",
    "estArrivalAirport": "estArrivalAirport",
    "estDepartureAirportHorizDistance": "estDepartureAirportHorizDistance",
    "estDepartureAirportVertDistance": "estDepartureAirportVertDistance",
    "estArrivalAirportHorizDistance": "estArrivalAirportHorizDistance",
    "estArrivalAirportVertDistance": "estArrivalAirportVertDistance",
    "departureAirportCandidatesCount": "departureAirportCandidatesCount",
    "arrivalAirportCandidatesCount": "arrivalAirportCandidatesCount",
}

# Durée de récupération (minutes)
step = 60

# date initiale
initial_date_str = "2025-01-01 01:00:00"

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)


start = int(initial_date.timestamp())
end = int(end_date.timestamp())




In [8]:
# Send OpenSky data to Kafka

for i in range(24*51):
    print(f"{start}-{end}")
    send_opensky_to_kafka("opensky-flights", OPENSKY_ALL_FLIGHTS_URL.format(start, end), USERNAME, PASSWORD)
    time.sleep(1)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = int(initial_date.timestamp())
    end = int(end_date.timestamp())

1735714800-1735718400
Sent 446 records.
1735718400-1735722000
Sent 445 records.
1735722000-1735725600
Sent 440 records.
1735725600-1735729200
Sent 433 records.
1735729200-1735732800
Sent 517 records.
1735732800-1735736400
Sent 527 records.
1735736400-1735740000
Sent 575 records.
1735740000-1735743600
Sent 633 records.
1735743600-1735747200
Sent 763 records.
1735747200-1735750800
Sent 748 records.
1735750800-1735754400
Sent 826 records.
1735754400-1735758000
Sent 813 records.
1735758000-1735761600
Sent 826 records.
1735761600-1735765200
Sent 758 records.
1735765200-1735768800
Sent 795 records.
1735768800-1735772400
Sent 1220 records.
1735772400-1735776000
Failed to fetch data: 404
1735776000-1735779600
Sent 3145 records.
1735779600-1735783200
Sent 800 records.
1735783200-1735786800
Sent 658 records.
1735786800-1735790400
Sent 633 records.
1735790400-1735794000
Sent 572 records.
1735794000-1735797600
Sent 589 records.
1735797600-1735801200
Sent 587 records.
1735801200-1735804800
Sent 567

ConnectionError: HTTPSConnectionPool(host='opensky-network.org', port=443): Max retries exceeded with url: /api/flights/all?begin=1737982800&end=1737986400 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f93d93474a0>: Failed to resolve 'opensky-network.org' ([Errno -3] Temporary failure in name resolution)"))